In [79]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from math import ceil

# Maybe: Print all of this code into Python 
# code to be used or not used by others
# Probably should keep the function names.
class Neat:
    
    def __init__(self, df, targetY, indexColumns=[]):        
        self.df = df        
        self.targetY = self._cleanColumnName(targetY)        
        self.indexColumns = self._cleanColumnNamesArray(indexColumns)     
        self.uniqueTargetYValues = self.df[self.targetY].unique()
        self.targetYMappings = {}
        self.numberColumns = []
        self.categoryColumns = []
        self.datetimeColumns = []
        self.medians = []
        self.lowerBounds = []
        self.upperBounds = []
        self.uniqueCategoryValues = {}
        self.valuesThatMapTo_Other = {}
        self.categoryFrequencies = {}
        self.targetYFrequencies = {}
        self.idealTargetYFrequencies = {}
        # TargetY
        self._setTargetYMappings()
        self._convertTargetYToNumeric()
        self._dropNATargetYRows()        
        # Column Metadata
        self._cleanColumnNamesDF()        
        self._setColumnDataTypes()        
        # Index
        self._dropDuplicatesAndMissingRowsIfIndexIsSpecified()    
        self._addIndex()
        # Numbers
        self._saveMediansAndBounds()
        self._fixMissingNumValuesAndInfinity()
        self._fixHighLeveragePoints()
        # Categories
        self._saveUniqueCategoryValues()   
        self._saveCategoryFrequenciesAndValuesThatMapTo_Other()
        self._dropCategoryColumnsWithAllMissingValues()
        self._fixMissingCategoryValuesAndMapValuesTo_Other()
        self._applyDummyEncoding()
        # Datetimes
        self._dropColumnsWithMissingDatetimeValues()
        self._convertDatetimeToNumber()
        # Class Imbalance
        self._saveTargetYFrequencies()
        self._saveIdealTargetYFrequencies()
        self._fixTargetYImbalance()
    
    def _cleanColumnNamesArray(self, indexColumns):
        if type(indexColumns) == str:
            indexColumns = [indexColumns]
        arr = []
        for column in indexColumns:
            arr.append(self._cleanColumnName(column))
        return arr
    
    def _cleanColumnName(self, string):
        return string.strip().lower().replace(' ', '_')
    
    ########## TargetY ##########
    
    def _setTargetYMappings(self):
        if self.df[self.targetY].dtype == 'object': # is a string
            i = 0      
            for value in self.uniqueTargetYValues:
                if value != None and value.strip() != "":
                    self.targetYMappings[value] = i
                    i = i + 1

    def _convertTargetYToNumeric(self):
        if self.df[self.targetY].dtype == 'object': # is a string        
            self.df[self.targetY] = self.df[self.targetY].map(self.targetYMappings)
    
    def _dropNATargetYRows(self):    
        rowsToDrop = []
        for i, row in self.df.iterrows():    
            rowsToDrop.append(i) if np.isnan(row[self.targetY]) else None            
        self.df = self.df.drop(self.df.index[rowsToDrop])    
    
    ########## Column Metadata ##########
    
    def _cleanColumnNamesDF(self):
        self.df.columns = self.df.columns.str.strip().str.lower().str.replace(' ', '_')    
        
    def _setColumnDataTypes(self):
        columns = self.df.columns.values.tolist()
        for column in columns:    
            if column == self.targetY or column in indexColumns:
                pass
            elif self.df[column].dtype == 'int64' or self.df[column].dtype == 'float64':
                self.numberColumns.append(column)
            elif self.df[column].dtype == 'object':
                self.categoryColumns.append(column)
            else:
                self.datetimeColumns.append(column)     
    
    ########## Index ##########
    
    def _dropDuplicatesAndMissingRowsIfIndexIsSpecified(self):
        rowsToDrop = []
        if self.indexColumns != []:
            self.df = self.df.drop_duplicates(subset=self.indexColumns)
            for i, row in self.df.iterrows(): 
                for column in self.indexColumns:
                    if ((self.df[column].dtype == 'int64' or self.df[column].dtype == 'float64') and (np.isnan(row[column]) or np.isinf(row[column]))) or row[column] == None:
                        rowsToDrop.append(i)
        self.df = self.df.drop(self.df.index[rowsToDrop])    
        
    def _addIndex(self):
        if self.indexColumns == []:
            self.df = self.df.set_index(np.arange(1,len(self.df.index)+1))
        else:
            self.df = self.df.set_index(self.indexColumns)                    
        
    ########## Numbers ##########
        
    def _saveMediansAndBounds(self):        
        firstQuantiles = self.df.quantile(.25)
        thirdQuantiles = self.df.quantile(.75)
        
        self.medians = self.df.quantile(.50)
        self.lowerBounds = {}
        self.upperBounds = {}
        for column in self.numberColumns:            
            self.lowerBounds[column] = self.medians[column] - 2*(self.medians[column] - firstQuantiles[column])
            self.upperBounds[column] = self.medians[column] + 2*(thirdQuantiles[column] - self.medians[column])        
        
    def _fixMissingNumValuesAndInfinity(self):
        self.df = self.df.fillna(self.medians) # optionally: replace self.medians with 0
        self.df.replace([np.inf, -np.inf], np.nan)
        self.df = self.df.fillna(self.upperBounds)        
        
    def _fixHighLeveragePoints(self):
        for i, row in self.df.iterrows(): 
            for column in self.numberColumns:
                if row[column] > self.upperBounds[column]:
                    self.df.at[i, column] = self.upperBounds[column]
                if row[column] < self.lowerBounds[column]:
                    self.df.at[i, column] = self.lowerBounds[column]
      
    ########## Categories ##########
    
    def _saveUniqueCategoryValues(self):        
        for column in self.categoryColumns:
            self.uniqueCategoryValues[column] = []
            for value in self.df[column].unique():
                if value == None:
                    continue    
                self.uniqueCategoryValues[column].append(value)
            self.uniqueCategoryValues[column].append('_Other')            
            
    def _saveCategoryFrequenciesAndValuesThatMapTo_Other(self):
        for column in self.categoryColumns: 
            _otherFrequency = 0
            self.valuesThatMapTo_Other[column] = []
            frequencyPercentage = pd.value_counts(self.df[column].values, sort=False, normalize=True)
            self.categoryFrequencies[column] = {}
            for value in self.uniqueCategoryValues[column]: 
                if value == '_Other':
                    continue
                elif frequencyPercentage[value] < .05: 
                    self.valuesThatMapTo_Other[column].append(value)
                    _otherFrequency = _otherFrequency + frequencyPercentage[value]
                else:
                    self.categoryFrequencies[column][value] = frequencyPercentage[value]            
            self.categoryFrequencies[column]['_Other'] = _otherFrequency
                    
    def _dropCategoryColumnsWithAllMissingValues(self):
        columnsToRemove = []
        for column in self.categoryColumns:
            if len(self.uniqueCategoryValues[column]) == 1 and self.uniqueCategoryValues[column][0] == '_Other':
                columnsToRemove.append(column)
        self._dropCategoryColumns(columnsToRemove)
        
    def _dropCategoryColumns(self, columnsToRemove):
        self.df = self.df.drop(columnsToRemove, 1)         
        for column in columnsToRemove:
            self.categoryColumns.remove(column) 
            
    def _fixMissingCategoryValuesAndMapValuesTo_Other(self):
        for i, row in self.df.iterrows(): 
            for column in self.categoryColumns:        
                if row[column] == None:
                    self.df.at[i, column] = self._getRandomCategoryBasedOnFrequencies(column)
                elif row[column] in self.valuesThatMapTo_Other[column]:
                    self.df.at[i, column] = '_Other'
            
    def _getRandomCategoryBasedOnFrequencies(self, column):
        chosenValue, prevValue, cumulativeProbability = None, None, 0
        randomNumber = np.random.uniform(0,1,1)[0]
        for value in self.uniqueCategoryValues[column]:
            probabilityOfValue, prevValue = self.categoryFrequencies[column][value], value
            cumulativeProbability = cumulativeProbability + probabilityOfValue
            if cumulativeProbability > randomNumber:
                chosenValue = value
                break
        return prevValue if chosenValue == None else chosenValue            

    def _applyDummyEncoding(self): # drop_first => dummy encoding instead of one hot encoding
        for column in self.categoryColumns:
            self.df = pd.concat([self.df.drop(column, axis=1), pd.get_dummies(self.df[column], prefix=column, drop_first=True)], axis=1)            

    ########## Datetimes ##########
            
    def _dropColumnsWithMissingDatetimeValues(self):
        columnsToDrop = []
        for i, row in self.df.iterrows(): 
            for column in self.datetimeColumns:
                if pd.isnull(row[column]) or row[column] == None:
                    columnsToDrop.append(column)
                    break
        self.df = self.df.drop(columnsToDrop, axis=1)
    
    def _convertDatetimeToNumber(self):        
        self.df[self.datetimeColumns] = self.df[self.datetimeColumns].apply(pd.to_numeric)                       
        
    ########## Class Imbalance ##########
        
    def _saveTargetYFrequencies(self):           
        self.targetYFrequencies = pd.value_counts(self.df[self.targetY].values, sort=True, normalize=False)        
        
    def _saveIdealTargetYFrequencies(self): 
        maxValue = None
        for value in self.uniqueTargetYValues:
            if maxValue == None or value > maxValue:
                maxValue = value
        
        minValue = ceil(maxValue / 2)
        for value in self.uniqueTargetYValues:
            actualFrequency = self.targetYFrequencies[value]
            self.idealTargetYFrequencies[value] = minValue if actualFrequency < minValue else actualFrequency
        
    def _fixTargetYImbalance(self):        
        pass
    
df = pd.DataFrame({'col1': ['a','b','c','a','z','g','b','a','i','t'], 'col2': [None,None,None,9,5,10,11,12,13,14]
                  , 'col3': ['test1','test1','test1','test1',None,None,'test1','test1','test2','test2']
                  , 'col4': [None, 5, 3 ,6 ,8, 10, 14, 87, 999 ,9999]
                  , 'col5': [None,None,None,None,'adsf','bas',None,None,None,None]})                
targetY = 'col1'
#indexColumns = 'col2'
indexColumns = ['col4']

neat = Neat(df, targetY, indexColumns)

neat.df
    
print(neat.targetYFrequencies)

for value in neat.targetYFrequencies:
    print(value)
for value in neat.targetYMappings:
    print(neat.targetYMappings[value])
print(neat.targetYMappings)
#print(neat.targetYMappings)
neat.df

    
    


1    2
0    2
6    1
5    1
4    1
3    1
2    1
dtype: int64
2
2
1
1
1
1
1
0
1
2
3
4
5
6
{'a': 0, 'b': 1, 'c': 2, 'z': 3, 'g': 4, 'i': 5, 't': 6}


,col1,col2,col3_test2,col5_bas
col4,,,,
5.0,1,11.0,0,1
3.0,2,11.0,0,1
6.0,0,9.0,0,0
8.0,3,8.0,1,0
10.0,4,10.0,0,1
14.0,1,11.0,0,0
87.0,0,12.0,0,1
999.0,5,13.0,1,1
9999.0,6,14.0,1,1


In [104]:
df = pd.DataFrame({'col1': [1,1, 2,3,4,5], 'col2': [3, 3,None, None, None,None]})
df
df.columns.values.tolist()
df.dtypes

    
       
        
print(numberColumns)        
print(categoryColumns)        
print(datetimeColumns)        


['col1', 'col2']
['col1', 'col2']
[]
[]


In [181]:
df = pd.DataFrame({'col1': [1,2,None],'col2': [2,4,None]})
#print(df)


#print(df.quantile(.25))
#print(df.quantile(.5))
#print(df.quantile(.75))
print(df.quantile(.5))

df = df.fillna(df.median())




print("A")
print(df)

col1    1.5
col2    3.0
Name: 0.5, dtype: float64
A
   col1  col2
0   1.0   2.0
1   2.0   4.0
2   1.5   3.0


In [144]:
round(2.50)

2

In [190]:
df.median()

col1     6.0
col2    11.0
dtype: float64

In [206]:
quantile1 = df.quantile(.25)
quantile3 = df.quantile(.75)


medians = df.quantile(.50)
print(quantile1)
print(medians)

# for col in medians:
#     print(col)
    





col1    4.5
col2    9.5
Name: 0.25, dtype: float64
col1     6.0
col2    11.0
Name: 0.5, dtype: float64
10.5
20.5


In [26]:
a = ["a"]
if a == []:
    print("HI")

In [37]:
type(['a'])
type('a')==str

True

In [18]:
df = pd.DataFrame({'col1': [1,1, 2,3,4,5], 'col2': ['a', 'b',None, None, None,None]})

# for cat in ['col2','col1']:
#     print("Levels for catgeory '{0}': {1}".format(cat, df[cat].unique()))

# df['col2']=df['col2'].map({'a':0,'b':1})

# df_class = df['col2'].values
# df_class

# for cat in ['col2']:
#     print(cat)
    
i = 0
targetMappings = {}
for value in df['col2'].unique():
    if value != None and value.strip() != "":
        targetMappings[value] = i
        i = i + 1
print(targetMappings)


df['col2'] = df['col2'].map(targetMappings)
df


{'a': 0, 'b': 1}


,col1,col2
0,1,0.0
1,1,1.0
2,2,NaN
3,3,NaN
4,4,NaN
5,5,NaN


In [8]:
import pandas as pd
import numpy as np
df = pd.DataFrame({'col1': [5,None,None,None,None,None], 'col2': ['a','a','b','b','b','c']})
uniqueCategoryValues = {}
categoryFrequencies = {}
# rowsToDrop = []
# for i, row in df.iterrows():    
#     rowsToDrop.append(i) if np.isnan(row['col2']) else None            
# df = df.drop(df.index[rowsToDrop])
# df
# a = 'col2'
#print(df.a.value_counts(normalize=True))
#print(pd.value_counts(df['col3'].values, sort=False, normalize=True))
uniqueCategoryValues['col2'] = df['col2'].unique()
#print(uniqueCategoryValues)
categoryFrequencies['col2'] = pd.value_counts(df['col2'].values, sort=False, normalize=True)
#print(categoryFrequencies['col2']['a'])
print(len(uniqueCategoryValues['col2']))
uniqueCategoryValues['col2'][0]
uniqueCategoryValues['col2']
for value in uniqueCategoryValues['col2']:
    if value == None:
        continue
    print(value)
print(categoryFrequencies['col2']['a'])
categoryFrequencies

columns = df.columns.values.tolist()  
columns
for column in columns:
    print(df[column].dtype)

3
a
b
c
0.333333333333
float64
object


In [90]:
counts = pd.value_counts(df['col2'].values, sort=False, normalize=True)


categoryFrequencies['col2'] = {}
for value in uniqueCategoryValues['col2']:
    if value == None:
        continue    
    categoryFrequencies['col2'][value] = counts[value]
        
categoryFrequencies['col2']



{'a': 0.33333333333333331, 'b': 0.5, 'c': 0.16666666666666666}

In [1]:
for value in df['col2'].unique():
    print(value)
  

NameError: name 'df' is not defined

In [2]:
columns = df.columns.values.tolist()
for column in columns:    
    if column == self.targetY or column in indexColumns:
        pass
    elif self.df[column].dtype == 'int64' or self.df[column].dtype == 'float64':
        self.numberColumns.append(column)
    elif df[column].dtype == 'object':
        self.categoryColumns.append(column)
    else:
        self.datetimeColumns.append(column)

NameError: name 'df' is not defined

In [2]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
df = pd.DataFrame({'col1': [5,None,None,None,None,None], 'col2': ['a','a','b','b','b','c']})
df2 = pd.DataFrame({'col1': [1,2,4], 'col2': ['b','b','c']})
#enc = preprocessing.OneHotEncoder()
#enc.fit()
#enc.fit(df2['col1'])

# df2 = pd.concat(
#     [
#         df2,
#         pd.get_dummies(df2['col2'], prefix='col2', drop_first=True)


df2 = pd.concat([df2.drop('col2', axis=1), pd.get_dummies(df2['col2'], prefix='col2', drop_first=True)], axis=1)
df2

,col1,col2_c
0,1,0
1,2,0
2,4,1


In [52]:
df = pd.DataFrame({'year': [2015, 2016, 2018],
                       'month': [2, 3, 4],
                       'day': [4, 5, 6]})
datetime = pd.to_datetime(df)
df['datetime'] = datetime
df['datetime2'] = datetime
df = df.drop(['year','month','day'], axis=1)

datetimeColumns = ['datetime','datetime2']

#df[~df.isnull()] = None
print(df.where(df.notnull(), None))


columnsToDrop = []
for i, row in df.iterrows(): 
    for column in datetimeColumns:
        if row[column] == None or pd.isnull(row[column]):
            columnsToDrop.append(column)
            break
df = df.drop(columnsToDrop,axis=1)

#pd.notnull(df['datetime'])
df
#df

#df[df.b.isnull()]


df[datetimeColumns] = df[datetimeColumns].apply(pd.to_numeric)
df

    datetime  datetime2
0 2015-02-04 2015-02-04
1 2016-03-05 2016-03-05
2 2018-04-06 2018-04-06


,datetime,datetime2
0,1423008000000000000,1423008000000000000
1,1457136000000000000,1457136000000000000
2,1522972800000000000,1522972800000000000


In [86]:
from math import ceil
ceil(2.1)

3